In [1]:
from dotenv import load_dotenv 
load_dotenv()

True

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'income_tax_collection',
    persist_directory = './income_tax_collection'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: list
    answer: str
    
graph_builder = StateGraph(AgentState)

In [4]:
def retrieve(state: AgentState) -> AgentState:
    """
    사용자의 질문에 기반하여 벡터 스토어에서 관련 문서를 검색합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 검색된 문서가 추가된 state를 반환합니다.
    """
    query = state['query']
    docs = retriever.invoke(query)
    return {'context': docs}

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
from langchain import hub

# 허브에서 RAG 프롬프트를 가져옵니다
generate_prompt = hub.pull("rlm/rag-prompt")

# 지정된 매개변수로 언어 모델을 초기화합니다
generate_llm = ChatOpenAI(model='gpt-4o', max_completion_tokens=100)

def generate(state: AgentState) -> AgentState:
    """
    주어진 state를 기반으로 RAG 체인을 사용하여 응답을 생성합니다.

    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 생성된 응답을 포함하는 state를 반환합니다.
    """
    context = state['context']
    query = state['query']
    
    rag_chain = generate_prompt | generate_llm
    
    response = rag_chain.invoke({'question': query, 'context': context})
    
    return {'answer': response.content}

/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
from typing import Literal
doc_relevance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relevance(state: AgentState) -> Literal['relevant', 'irrelvant']:
    """
    주어진 state를 기반으로 문서의 관련성을 판단합니다.

    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state.

    Returns:
        Literal['relevant', 'irrelevant']: 문서가 관련성이 높으면 'relevant', 그렇지 않으면 'irrelevant'를 반환합니다.
    """
    query = state['query']
    context = state['context']

    doc_relevance_chain = doc_relevance_prompt | llm
    response = doc_relevance_chain.invoke({'question': query, 'documents': context})

    if response['Score'] == 1:
        return 'relevant'
    
    return 'irrelvant'

/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

rewrite_prompt = PromptTemplate.from_template("""사용자의 질문을 보고, 웹 검색에 용이하게 사용자의 질문을 수정해주세요
질문: {query}
""")

def rewrite(state: AgentState):
    """
    사용자의 질문을 사전을 참고하여 변경합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 변경된 질문을 포함하는 state를 반환합니다.
    """
    query = state['query']
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()

    response = rewrite_chain.invoke({'query': query})
    print('rewrite:', response)

    return {'query': response}


In [9]:
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

def web_search(state: AgentState) -> AgentState:
    """
    주어진 state를 기반으로 웹 검색을 수행합니다.

    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state.

    Returns:
        AgentState: 웹 검색 결과가 추가된 state를 반환합니다.
    """
    query = state['query']
    results = tavily_search_tool.invoke(query)

    print('web_search: ', results)

    return {'context': results}

In [10]:
graph_builder.add_node('retrieve', retrieve)
graph_builder.add_node('generate', generate)
graph_builder.add_node('rewrite', rewrite)
graph_builder.add_node('web_search', web_search)

In [11]:
# from langgraph.graph import START, END 

# graph_builder.add_edge(START, 'retrieve')
# graph_builder.add_conditional_edges(
#     'retrieve',
#     check_doc_relevance,
#     {
#         'relevant': 'generate',
#         'irrelvant': 'rewrite'
#     }
# )
# graph_builder.add_edge('rewrite', 'web_search')
# graph_builder.add_edge('web_search', 'generate')
# graph_builder.add_edge('generate', END)

In [12]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'retrieve')
graph_builder.add_conditional_edges(
    'retrieve',
    check_doc_relevance,
    {
        'relevant': 'generate',
        'irrelvant': 'web_search'
    }
)
# graph_builder.add_edge('rewrite', 'web_search')
graph_builder.add_edge('web_search', 'generate')
graph_builder.add_edge('generate', END)

In [13]:
graph = graph_builder.compile()

In [14]:
graph.get_graph().print_ascii()


            +-----------+       
            | __start__ |       
            +-----------+       
                   *            
                   *            
                   *            
             +----------+       
             | retrieve |       
             +----------+       
            ...         ...     
           .               .    
         ..                 ... 
+------------+                 .
| web_search |              ... 
+------------+             .    
            ***         ...     
               *       .        
                **   ..         
             +----------+       
             | generate |       
             +----------+       
                   *            
                   *            
                   *            
              +---------+       
              | __end__ |       
              +---------+       


In [15]:
# query = '연봉 5천만원 거주자의 소득세는 얼마인가요?'

# initial_state = {
#     'query' : query
# }

# context = retrieve(initial_state)['context']

# relevance_state = {
#     'query': query,
#     'context': context
# }

# check_doc_relevance(relevance_state)

# generate_state = {
#     'context': context,
#     'query': query
# }

# generate(generate_state)

In [16]:
query = '연봉 5천만원 거주자의 소득세는 얼마인가요?'

initial_state = {
    'query' : query
}

graph.invoke(initial_state)


{'query': '연봉 5천만원 거주자의 소득세는 얼마인가요?',
 'context': [Document(metadata={'source': './documents/income_tax.txt'}, page_content='| 1,400만원 초과     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)                    |\n| 5,000만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)                  |\n| 8,800만원 이하     | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)                |\n| 1억5천만원 이하    | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)                |\n| 3억원 이하        | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)                    |\n| 5억원 이하        | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)                 |\n| 10억원 이하       | 3억3,406만원 + (10억원을 초과하는 금액의 45퍼센트)                |\n법제처 35 국가법령정보센터\n소득세법\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 "퇴직소득 산출세액"이라 한다)으로 한다. <개정 2013. 1. 1, 2014. 12. 23.>\n   1. 해당 과세기간의 퇴직소득과세표준에 제11항의 세율을 적용하여 계산한 금액\n   2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n   3. 삭제 <2014. 12. 23.>\n   [전문개정 2009. 12. 31.]\n제2관 세액공제 <개정 2009. 12. 31.>\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 

In [17]:
query = '문정역 맛집 추천'

initial_state = {
    'query' : query
}

graph.invoke(initial_state)


web_search:  [{'url': 'https://hotel-iu.tistory.com/3719', 'content': "문정역맛집 베스트 추천 top 10문정역맛집 베스트 추천 top 10곳을 소개합니다. 1: 꽂따 문정본점상호명: 꽂따 문정본점주소: 서울특별시 송파구 문정동 642-3 문정에스케이브이원지엘메트로시티 C동 108, 109호전화번호: 미입력관련 키워드: ['문정역치킨', '문정역치킨집', '문정역수제맥주', '문정역회식"}, {'url': 'https://maimiublog.com/서울-문정역-맛집-추천-best-5-문정역-술집추천-문정동-가/', 'content': '서울 문정역 맛집 추천, 술집 Best 5 1. 골목떡볶이 2. 라치오 3. 더부엌 4. 로바타 나미 5. 삼촌네포차 0. 연인에게 주는 특별한 기념일 선물! 서울 문정역 근처 가볼만한곳 1. 강남실탄사격장 송파파크하비오본점 2. 송파파크하비오워터킹덤워터파크&찜질스파 3. 문'}, {'url': 'https://seo15647.tistory.com/entry/문정역-맛집-베스트10-정리', 'content': '오늘 포스팅에서는 문정역 맛집 추천장소 알아볼께요 문정역 맛집 베스트10 추천 정보 1. 툇마루밥상 정보 주소 : 서울 송파구 송파대로22길 11 비교적 저렴한 가격에 한정식을 맛볼 수 있는 곳으로, 일반 가정집을 고쳐 식당으로 이용하고 있다.'}]


{'query': '문정역 맛집 추천',
 'context': [{'url': 'https://hotel-iu.tistory.com/3719',
   'content': "문정역맛집 베스트 추천 top 10문정역맛집 베스트 추천 top 10곳을 소개합니다. 1: 꽂따 문정본점상호명: 꽂따 문정본점주소: 서울특별시 송파구 문정동 642-3 문정에스케이브이원지엘메트로시티 C동 108, 109호전화번호: 미입력관련 키워드: ['문정역치킨', '문정역치킨집', '문정역수제맥주', '문정역회식"},
  {'url': 'https://maimiublog.com/서울-문정역-맛집-추천-best-5-문정역-술집추천-문정동-가/',
   'content': '서울 문정역 맛집 추천, 술집 Best 5 1. 골목떡볶이 2. 라치오 3. 더부엌 4. 로바타 나미 5. 삼촌네포차 0. 연인에게 주는 특별한 기념일 선물! 서울 문정역 근처 가볼만한곳 1. 강남실탄사격장 송파파크하비오본점 2. 송파파크하비오워터킹덤워터파크&찜질스파 3. 문'},
  {'url': 'https://seo15647.tistory.com/entry/문정역-맛집-베스트10-정리',
   'content': '오늘 포스팅에서는 문정역 맛집 추천장소 알아볼께요 문정역 맛집 베스트10 추천 정보 1. 툇마루밥상 정보 주소 : 서울 송파구 송파대로22길 11 비교적 저렴한 가격에 한정식을 맛볼 수 있는 곳으로, 일반 가정집을 고쳐 식당으로 이용하고 있다.'}],
 'answer': '문정역 맛집 추천으로는 꽂따 문정본점, 골목떡볶이, 라치오, 더부엌, 로바타 나미, 삼촌네포차, 그리고 툇마루밥상이 있습니다.'}